In [1]:
from bs4 import BeautifulSoup as BS
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import schedule
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseUpload
from datetime import datetime
import io
import time

In [2]:
# Set up credentials
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'service_account.json'

creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Set up the Drive API
drive_service = build('drive', 'v3', credentials=creds)

In [3]:
def scrape():

    #hide webdriver
    options = Options()
    options.add_argument('--headless')

    #webdriver to scrape html from loaded js iframe
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = options)
    driver.get("https://stats.vsinstats.com/pageviewer/vsinstats.php?pageid=vsindksplits")
    #time.sleep(5)
    #driver.switch_to.frame("offer_22b0fe8e932ca85c592e-0")
    #driver.find_element(By.CLASS_NAME, 'pn-template__close-btn').click()
    time.sleep(3)

    #iframe = driver.find_elements(By.XPATH, '//iframe')[0]
    #driver.switch_to.frame(iframe)
    soup = BS(driver.page_source)

    sports = {'NBA': 'dk-nba', 'MLB': 'dk-mlb' }

    for key, value in sports.items():
        
        #target specific iframe table html (nfl = "dk-nfl", ncaa = "dk-ncaaf", mlb = "dk-mlb", nba = "dk-nba", nhl = "dk-nhl")
        sport_table = soup.find(lambda tag: tag.name=='div' and tag.has_attr('id') and tag['id']==value) #replace w/ sport of choice
        table = sport_table.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="dksplits")

        #function that parses html to extract specific data from iframe table
        def tableDataText(table):
            def rowgetDataText(tr, coltag='td'):  # td (data) or th (header)
                return [td.get_text(strip=True) for td in tr.find_all(coltag)]

            rows = []
            trs = table.find_all('tr')
            header_rows = []
            data_rows = []

            for tr in trs:  # find all header rows and table rows
                ths = tr.find_all('th')
                tds = tr.find_all('td')
                if ths:
                    header_row = rowgetDataText(tr, 'th')
                    if len(header_rows) == 1:
                        rows.extend(data_rows)  # Append data rows before the second header row
                        data_rows = []  # Reset data rows
                        rows.append(header_row)  # Append the second header row
                    else:
                        header_rows.append(header_row)  # Append the first header row
                elif tds:
                    if len(header_rows) == 1:
                        rows.append(rowgetDataText(tr, 'td'))
                    else:
                        data_rows.append(rowgetDataText(tr, 'td'))
            if data_rows:  # Append remaining data rows
                rows.extend(data_rows)
            if header_rows:  # Append header rows in chronological order
                rows.insert(0, header_rows[0])  # First header row
            return rows

        list_table = tableDataText(table)
        date = list_table[0][0]
        spread_type = list_table[0][1] #grab spread, run line, goal line

        #make df from html and drop null rows
        dftable = pd.DataFrame(list_table[1:], columns=list_table[0])
        dftable = dftable.dropna()

        #rename columns
        cols = []
        count = 1
        for column in dftable.columns:
            if column == "% Bets":
                cols.append(count)
                count+=1
            elif column == "% Handle":
                cols.append(count)
                count+=1
            else:
                cols.append(column)
        dftable.columns = cols
        dftable.rename(columns={ dftable.columns[2]: f"% {spread_type} Handle", dftable.columns[3]: f"% {spread_type} Bets", 
                                dftable.columns[5]: "% O/U Handle", dftable.columns[6]: "% O/U Bets", 
                                dftable.columns[9]: "% Moneyline Bets", dftable.columns[8]: "% Moneyline Handle" }, inplace = True)

        #clean data so we can still build appropriate df when they release partial lines (still needs work)
        for row in dftable[spread_type]:
            if '--' in row:
                dftable[spread_type] = dftable[spread_type].str.replace("--", "+0-0")

        for row in dftable['Total']:
            if '--' in row:
                dftable['Total'] = dftable['Total'].str.replace("--", "Ov 000Un 000")
            if '-' in row:
                dftable['Total'] = dftable['Total'].str.replace("-", "Ov 000")

        for row in dftable['Moneyline']:
            if '--' in row:
                dftable['Moneyline'] = dftable['Moneyline'].str.replace("--", "+000-000")


        #regex to extract and put the data in the correct order
        d1 = dftable[date].str.split("(?<=[a-z])(?=[A-Z])").explode()
        d1 = d1.str.split("(?<=\(G2\))(?=[A-Z])").explode()
        d2 = dftable[spread_type].str.findall(r'([+|-][0-9]*\.[0-9]|[+|-]\d+)').explode()
        d3 = dftable[f'% {spread_type} Handle'].str.findall(r'(\d+%)').explode()
        d4 = dftable[f'% {spread_type} Bets'].str.findall(r'(\d+%)').explode()
        d5 = dftable['Total'].str.findall(r'([A-Za-z ]+\d+)').explode()
        d6 = dftable['% O/U Handle'].str.findall(r'(\d+%)').explode()
        d7 = dftable['% O/U Bets'].str.findall(r'(\d+%)').explode()
        d8 = dftable['Moneyline'].str.findall(r'[+|-]\d+').explode()
        d9 = dftable['% Moneyline Handle'].str.findall(r'(\d+%)').explode()
        d10 = dftable['% Moneyline Bets'].str.findall(r'(\d+%)').explode()


        #build df
        data = { date : d1,
                spread_type : d2,
                f'% {spread_type} Handle': d3,
                f'% {spread_type} Bets': d4,
                'Total': d5,
                '% O/U Handle': d6,
                '% O/U Bets': d7,
                'Moneyline': d8,
                '% Moneyline Handle': d9,
                '% Moneyline Bets': d10 }

        try:
            df = pd.DataFrame(data)

            #add columns for spread and O/U sharp differential
            df[f"% {spread_type} Handle"] = df[f"% {spread_type} Handle"].str.rstrip("%").astype(int)
            df[f"% {spread_type} Bets"] = df[f"% {spread_type} Bets"].str.rstrip("%").astype(int)
            df["% O/U Handle"] = df["% O/U Handle"].str.rstrip("%").astype(int)
            df["% O/U Bets"] = df["% O/U Bets"].str.rstrip("%").astype(int)
            df['% Moneyline Handle'] = df['% Moneyline Handle'].str.rstrip("%").astype(int)
            df['% Moneyline Bets'] = df['% Moneyline Bets'].str.rstrip("%").astype(int)

            df[f"{spread_type} Sharp Differential"] = df[f"% {spread_type} Handle"] - df[f"% {spread_type} Bets"]
            df["O/U Sharp Differential"] = df["% O/U Handle"] - df["% O/U Bets"]
            df["Moneyline Sharp Differential"] = df["% Moneyline Handle"] - df["% Moneyline Bets"]

            df[f"{spread_type} Sharp Differential"] = df[f"{spread_type} Sharp Differential"].astype(str) + '%'
            df[f"% {spread_type} Handle"] = df[f"% {spread_type} Handle"].astype(str) + '%'
            df[f"% {spread_type} Bets"] = df[f"% {spread_type} Bets"].astype(str) + '%'
            df["O/U Sharp Differential"] = df["O/U Sharp Differential"].astype(str) + '%'
            df["% O/U Handle"] = df["% O/U Handle"].astype(str) + '%'
            df["% O/U Bets"] = df["% O/U Bets"].astype(str) + '%'
            df["Moneyline Sharp Differential"] = df["Moneyline Sharp Differential"].astype(str) + '%'
            df['% Moneyline Handle'] = df['% Moneyline Handle'].astype(str) + '%'
            df['% Moneyline Bets'] = df['% Moneyline Bets'].astype(str) + '%'
            
        except ValueError:
                    print("Table not ready, scrape again soon")

        # Add current timestamp
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Save data frame to buffer
        excel_buffer = io.BytesIO()
        writer = pd.ExcelWriter(excel_buffer, engine='xlsxwriter')
        df.to_excel(writer, sheet_name=key, index=False)
        writer.close()

        # Write data frame to Google Drive
        file_metadata = {
            'name': f'{key} {current_time}.xlxs',
            'parents': ['1UtpOjotbnUvoG0fHjunsA9vrRaAqulu9']  
        }

        # Write to google drive
        media = MediaIoBaseUpload(excel_buffer, mimetype='application/vnd.ms-excel')
        file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
        ).execute()
        print(f'File ID: {file.get("id")}')

    return

In [ ]:
schedule.every(10).minutes.do(scrape)

while True:
    schedule.run_pending()
    time.sleep(1)